In [1]:
import pandas as pd
import numpy as np
import re
from keras.layers import Dense, Dropout, GRU, Embedding, TimeDistributed, Flatten
from keras.layers import Input, Activation, concatenate, GlobalAveragePooling1D
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Dropout
from keras.layers import RepeatVector, Permute, merge, Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils, get_custom_objects
from keras.preprocessing import text, sequence
from string import ascii_letters, punctuation, digits
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Activation(swish)})

SEQ_LENGTH = 100
EMBED_SIZE = 100
VOCAB = 238590
np.random.seed(2017)

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### model

In [2]:
def define_model(matrix):
    rnn = {}
    rnn['units'] = 50
    rnn['return_sequences'] = True
    rnn['recurrent_dropout'] = 0.2
    rnn['dropout'] = 0.1
    rnn['activation'] = 'tanh'
    inputs = Input(shape=(SEQ_LENGTH,), name='sequence')
    embed = Embedding(VOCAB,EMBED_SIZE, weights=[matrix], trainable=False)(inputs)
    lstm = GRU(**rnn)(embed)
    atten = TimeDistributed(Dense(1, activation='tanh'))(lstm)
    atten = Flatten()(atten)
    atten = Activation('softmax')(atten)
    atten = RepeatVector(rnn['units'])(atten)
    atten = Permute([2, 1])(atten)
    dense = merge([lstm, atten], mode='mul')
    dense = Lambda(lambda xin: K.sum(xin, axis=1))(dense)
    dense = BatchNormalization()(dense)
    dense = Dense(200, activation='swish')(dense)
    dense = Dropout(0.2)(dense)
    predict = Dense(6, activation='sigmoid')(dense)
    model = Model(inputs=[inputs], output=predict)
    optimizer = Adam(lr=1e-3)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

### generator

In [3]:
embeddings_index = {}

f = open('../data/data/fasttext/vector.vec')

for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print(values[0])
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 93651 word vectors.


In [4]:
def dataflow(train_text, valid_text):
    train_text['comment_text'] = train_text['comment_text'].fillna('nan')
    valid_text['comment_text'] = valid_text['comment_text'].fillna('nan')
    train_text = list(train_text['comment_text'].values)
    valid_text = list(valid_text['comment_text'].values)
    tokenizer = text.Tokenizer(lower=True, char_level=False, num_words=100000)
    tokenizer.fit_on_texts(train_text + valid_text)
    word_index = tokenizer.word_index
    print('total words:', len(word_index))
    intersect = 0
    embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            intersect += 1
    print('common words:', intersect)
    train_token = tokenizer.texts_to_sequences(train_text)
    valid_token = tokenizer.texts_to_sequences(valid_text)
    train_seq = sequence.pad_sequences(train_token, maxlen=SEQ_LENGTH)
    valid_seq = sequence.pad_sequences(valid_token, maxlen=SEQ_LENGTH)
    return train_seq, valid_seq, embedding_matrix

def callbacks(suffix):
    stop = EarlyStopping('val_loss', patience=5, mode="min")
    path = '../data/data/source_1/model_3/model_{}.hdf5'.format(suffix)
    save = ModelCheckpoint(path, save_best_only=True, save_weights_only=True)
    logger = CSVLogger('../data/data/source_1/model_3/logger_{}.log'.format(suffix))
    reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, verbose=0, mode='min')
    return [stop, save, reduce, logger]

### data : fold - 1

In [5]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_1.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_1.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_1.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_1.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(1)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 [==============================] - 75s 586us/step - loss: 0.1305 - acc: 0.9621 - val_loss: 0.0943 - val_acc: 0.9717
Epoch 2/30
127656/127656 [==============================] - 73s 570us/step - loss: 0.0607 - acc: 0.9784 - val_loss: 0.0498 - val_acc: 0.9814
Epoch 3/30
127656/127656 [==============================] - 73s 575us/step - loss: 0.0530 - acc: 0.9806 - val_loss: 0.0485 - val_acc: 0.9818
Epoch 4/30
127656/127656 [==============================] - 73s 574us/step - loss: 0.0506 - acc: 0.9813 - val_loss: 0.0458 - val_acc: 0.9825
Epoch 5/30
127656/127656 [==============================] - 74s 580us/step - loss: 0.0492 - acc: 0.9817 - val_loss: 0.0446 - val_acc: 0.9830
Epoch 6/30
127656/127656 [==============================] - 72s 568us/step - loss: 0.0481 - acc: 0.9819 - val_loss: 0.0446 - val_acc: 0.9831
Epoch 7/30
127656/127656 [==============================] - 72s 564us/step - loss: 0.0469 - acc: 0.9824

### data : fold - 2 

In [6]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_2.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_2.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_2.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_2.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(2)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 68s 529us/step - loss: 0.1213 - acc: 0.9644 - val_loss: 0.0656 - val_acc: 0.9782
Epoch 2/30
127657/127657 [==============================] - 73s 570us/step - loss: 0.0557 - acc: 0.9799 - val_loss: 0.0496 - val_acc: 0.9817
Epoch 3/30
127657/127657 [==============================] - 73s 572us/step - loss: 0.0516 - acc: 0.9811 - val_loss: 0.0491 - val_acc: 0.9821
Epoch 4/30
127657/127657 [==============================] - 72s 568us/step - loss: 0.0494 - acc: 0.9818 - val_loss: 0.0461 - val_acc: 0.9826
Epoch 5/30
127657/127657 [==============================] - 73s 569us/step - loss: 0.0476 - acc: 0.9823 - val_loss: 0.0450 - val_acc: 0.9830
Epoch 6/30
127657/127657 [==============================] - 73s 573us/step - loss: 0.0466 - acc: 0.9826 - val_loss: 0.0444 - val_acc: 0.9830
Epoch 7/30
127657/127657 [==============================] - 73s 574us/step - loss: 0.0458 - acc: 0.9828

### data : fold - 3 

In [7]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_3.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_3.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_3.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_3.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(3)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 68s 530us/step - loss: 0.1108 - acc: 0.9675 - val_loss: 0.0615 - val_acc: 0.9788
Epoch 2/30
127657/127657 [==============================] - 73s 572us/step - loss: 0.0534 - acc: 0.9807 - val_loss: 0.0492 - val_acc: 0.9816
Epoch 3/30
127657/127657 [==============================] - 73s 571us/step - loss: 0.0501 - acc: 0.9815 - val_loss: 0.0466 - val_acc: 0.9828
Epoch 4/30
127657/127657 [==============================] - 73s 570us/step - loss: 0.0480 - acc: 0.9821 - val_loss: 0.0455 - val_acc: 0.9829
Epoch 5/30
127657/127657 [==============================] - 74s 576us/step - loss: 0.0466 - acc: 0.9825 - val_loss: 0.0451 - val_acc: 0.9831
Epoch 6/30
127657/127657 [==============================] - 73s 574us/step - loss: 0.0457 - acc: 0.9828 - val_loss: 0.0443 - val_acc: 0.9832
Epoch 7/30
127657/127657 [==============================] - 72s 568us/step - loss: 0.0450 - acc: 0.9829

### data : fold - 4

In [8]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_4.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_4.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_4.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_4.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(4)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 68s 532us/step - loss: 0.1047 - acc: 0.9699 - val_loss: 0.0557 - val_acc: 0.9794
Epoch 2/30
127657/127657 [==============================] - 73s 572us/step - loss: 0.0527 - acc: 0.9808 - val_loss: 0.0471 - val_acc: 0.9820
Epoch 3/30
127657/127657 [==============================] - 71s 559us/step - loss: 0.0494 - acc: 0.9818 - val_loss: 0.0459 - val_acc: 0.9825
Epoch 4/30
127657/127657 [==============================] - 73s 570us/step - loss: 0.0478 - acc: 0.9822 - val_loss: 0.0454 - val_acc: 0.9826
Epoch 5/30
127657/127657 [==============================] - 73s 572us/step - loss: 0.0465 - acc: 0.9825 - val_loss: 0.0442 - val_acc: 0.9831
Epoch 6/30
127657/127657 [==============================] - 71s 553us/step - loss: 0.0459 - acc: 0.9828 - val_loss: 0.0438 - val_acc: 0.9834
Epoch 7/30
127657/127657 [==============================] - 67s 525us/step - loss: 0.0451 - acc: 0.9831

### data : fold - 5

In [9]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_5.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_5.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_5.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_5.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(5)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 68s 532us/step - loss: 0.1082 - acc: 0.9681 - val_loss: 0.0656 - val_acc: 0.9768
Epoch 2/30
127657/127657 [==============================] - 73s 572us/step - loss: 0.0532 - acc: 0.9807 - val_loss: 0.0500 - val_acc: 0.9813
Epoch 3/30
127657/127657 [==============================] - 72s 563us/step - loss: 0.0496 - acc: 0.9816 - val_loss: 0.0488 - val_acc: 0.9820
Epoch 4/30
127657/127657 [==============================] - 73s 569us/step - loss: 0.0475 - acc: 0.9822 - val_loss: 0.0468 - val_acc: 0.9825
Epoch 5/30
127657/127657 [==============================] - 73s 571us/step - loss: 0.0460 - acc: 0.9827 - val_loss: 0.0460 - val_acc: 0.9827
Epoch 6/30
127657/127657 [==============================] - 67s 525us/step - loss: 0.0453 - acc: 0.9830 - val_loss: 0.0469 - val_acc: 0.9827
Epoch 7/30
127657/127657 [==============================] - 67s 525us/step - loss: 0.0445 - acc: 0.9833